In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle
from IPython.display import clear_output

In [ ]:
df = pd.read_csv(r"fcfe v2 tested (baseline).csv")
df["Date"]=pd.to_datetime(df["Date"], format='%Y-%m-%d')
df["Sector"] = df["Sector"].astype(str)
df["Group"] = df["Group"].astype(str)
df["Industry"] = df["Industry"].astype(str)
df["Subindustry"] = df["Subindustry"].astype(str)

In [7]:
df[df["Net Income"] == df["Net Income"].min()]
# AOL Time Warner listed on NYSE!!!!

,Unnamed: 0,GVKEY,LPERMNO,Date,Ticker,Company,Calendar Quarter,Fiscal Quarter,Book Value,Cost of Goods Sold,...,Sector,Subindustry,FCFE,Dividends,CPI_Multiplier,Market Cap,Adjusted_FCFE,Price_Exuberance,Adjusted_FCFE_Exuberance,Dividends_Exuberance
143071,143071,29241,79879,2001-03-31,VIAV,VIAVI SOLUTIONS INC,2001Q1,2001Q3,22134.4,449.2,...,45,45201020,-22570.300965,0.0,0.567859,13780.727747,0.1,1,1,3


Bubble criteria.

In [26]:
# here I define my bubble criteria
df["Bubble"] = 0
for permno, group in df.groupby("LPERMNO"):

    dates = group.loc[group["Price_Exuberance"]==1,"Date"]
    for i in dates:
        relevant_period = group[group.Date>=(i-pd.tseries.offsets.QuarterEnd(1))]
        # I allow prices to adjust to fundamentals in the previous quarter (which would only be reported in the current quarter)
        # i.e. prices should be forward looking
        # implicitly, I assume securities were not underpriced in the first place
        if (relevant_period["Adjusted_FCFE_Exuberance"]==0).all(): 
            # so no sign of explosive fundamentals, then it must be a bubble
            df.loc[(df.Date==i)&(df.LPERMNO ==permno),"Bubble"] =1

In [28]:
df["Change in Market Cap"] = df.groupby("LPERMNO")["Market Cap"].diff()

In [ ]:
dates = pd.date_range('30/06/1973','30/06/2005',freq = 'Q')
exuberance = pd.DataFrame({"Date":dates,"Total Change in Market Cap":np.zeros(len(dates))})

# getting total market cap
# for date, group in df.groupby("Date"):
# exuberance.loc[exuberance.Date ==date,"Total Market Cap"] = group["Market Cap"].sum() 
# total nasdaq market cap at this date
    
# creating sectoral columns to record exuberance
for i in df["Sector"].unique():
    exuberance[i]=np.zeros(len(dates))
    # exuberance[i+"_total"]=np.zeros(len(dates))
    
# getting exuberant market cap for sectors
for date_sector, group in df.groupby(["Date","Sector"]):
    exuberant_market_cap = group.loc[group["Price_Exuberance"]==1,"Market Cap"].sum()
    exuberance.loc[exuberance.Date ==date_sector[0],date_sector[1]] = exuberant_market_cap


In [ ]:
sector_dict = {"45":"Information Technology","50":"Communication Services","20":"Industrials",
              "25":"Consumer Discretionary","35":"Healthcare","30":"Consumer Staples","15":"Materials",
              "55":"Utilities","10":"Energy","60":"Real Estate","40":"Financials","Other":"Other"}

palette = cycle(px.colors.qualitative.Pastel+px.colors.qualitative.Bold)

bars =  []
for sector in df["Sector"].unique():
    bars.append(
        go.Bar(name = sector_dict[sector], x = exuberance["Date"], 
               y = exuberance[sector], marker_color = next(palette) 
                      )
    )

fig = go.Figure(data = bars)

fig.update_layout(barmode='stack')

#     fig.add_trace(
#         go.Bar(name = "Total Market Cap", x = exuberance["Date"], 
#                    y = exuberance["Total Market Cap"], marker_color = next(palette),opacity = 0.5)
#         )

fig.update_layout(
    title="NASDAQ Exuberance by Sector, 1973-2005, measured in market cap",
    xaxis_title="Date",
    yaxis_title="Total Market Cap of stocks in exuberance")
fig.show()

# need to use bar chart because you can't stack histograms easily to show sector composition.

In [37]:
df.loc[(df.Date>"2000-06-20")&(df.Sector=="45")&(df.Bubble==1),["Date","Company","Price","Bubble"]]

,Date,Company,Price,Bubble
623,2000-06-30,PLEXUS CORP,65.621370,1
1261,2002-06-30,SUN MICROSYSTEMS INC,2.789532,1
1262,2002-09-30,SUN MICROSYSTEMS INC,1.432522,1
1873,2000-12-31,MICROSOFT CORP,24.842497,1
1874,2001-03-31,MICROSOFT CORP,31.054798,1
...,...,...,...,...
217771,2004-06-30,VIDEO DISPLAY CORP,9.793542,1
217772,2004-09-30,VIDEO DISPLAY CORP,19.041096,1
217773,2004-12-31,VIDEO DISPLAY CORP,7.120501,1
230383,2002-06-30,UNITED ONLINE INC,6.692650,1
